In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [3]:
weather_df = pd.read_csv('../data/internal/weather.csv')

In [4]:
weather_df = weather_df.drop("Country", axis='columns')
weather_df = weather_df.drop("Name", axis='columns')

In [5]:
weather_df["Weather"].unique()

array(['Clear', 'Haze', 'Mist', 'Clouds', 'Snow', 'Thunderstorm', 'Rain',
       'Drizzle', 'Fog', 'Smoke'], dtype=object)

In [6]:
weather_df

,Weather,Temp,Humidity,Visibility,Wind speed,Clouds
0,Clear,280.77,57,10000,3.60,0
1,Haze,298.99,83,6000,2.06,20
2,Mist,293.25,83,1500,0.00,0
3,Clouds,288.07,47,10000,5.16,27
4,Mist,300.18,83,2200,3.09,20
...,...,...,...,...,...,...
3503,Clouds,280.24,50,10000,2.68,19
3504,Clear,285.26,56,10000,4.12,0
3505,Clouds,297.75,66,10000,1.02,56
3506,Clear,297.39,70,10000,3.74,6


In [7]:
# Thay đổi nhãn của các loại thời tiết không thuộc Clear hoặc Clouds thành 'Thời tiết xấu'
weather_df['Weather'] = weather_df['Weather'].replace(['Haze', 'Mist', 'Snow', 'Thunderstorm', 'Rain', 'Drizzle', 'Fog', 'Smoke'], 'Poor weather')

# Lấy các feature và nhãn đã chỉnh sửa
features = weather_df[['Temp', 'Humidity', 'Visibility', 'Wind speed', 'Clouds']]
labels = weather_df['Weather']

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Tạo mô hình Random Forest và huấn luyện nó
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # Số cây quyết định = 100
rf_model.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
predictions = rf_model.predict(X_test)

# Đánh giá mô hình
precision = precision_score(y_test, predictions, average='weighted')
accuracy = accuracy_score(y_test, predictions)
recall = recall_score(y_test, predictions, average='weighted')
print(f'Precision: {precision * 100:.2f}%')
print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'Recall: {recall * 100:.2f}%')

Precision: 90.50%
Accuracy: 91.03%
Recall: 91.03%


### GridSearchCV (Grid Search Cross-Validation): 
- Is a technique used for hyperparameter tuning in machine learning. The goal of hyperparameter tuning is to find the best set of hyperparameters for a machine learning model, which results in the optimal performance on a specific task or dataset.

sklearn.model_selection.GridSearchCV(estimator, param_grid, *, scoring=None, n_jobs=None, refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)

In [14]:
# Thay đổi nhãn của các loại thời tiết không thuộc Clear hoặc Clouds thành 'Thời tiết xấu'
weather_df['Weather'] = weather_df['Weather'].replace(['Haze', 'Mist', 'Snow', 'Thunderstorm', 'Rain', 'Drizzle', 'Fog', 'Smoke'], 'Poor weather')

# Lấy các feature và nhãn đã chỉnh sửa
features = weather_df[['Temp', 'Humidity', 'Visibility', 'Wind speed', 'Clouds']]
labels = weather_df['Weather']

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': ['balanced', 'balanced_subsample', None]
}

# Create the Random Forest classifier
rf_model = RandomForestClassifier(random_state=42)

# Create GridSearchCV object
grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='accuracy')

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters
best_params = grid_search.best_params_
print(f'Best Hyperparameters: {best_params}')

# Get the best model
best_rf_model = grid_search.best_estimator_

# Make predictions on the test set using the best model
best_predictions = best_rf_model.predict(X_test)

# Evaluate the best model
best_precision = precision_score(y_test, best_predictions, average='weighted')
best_accuracy = accuracy_score(y_test, best_predictions)
best_recall = recall_score(y_test, best_predictions, average='weighted')

print(f'Best Precision: {best_precision * 100:.2f}%')
print(f'Best Accuracy: {best_accuracy * 100:.2f}%')
print(f'Best Recall: {best_recall * 100:.2f}%')

Best Hyperparameters: {'class_weight': None, 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}
Best Precision: 90.85%
Best Accuracy: 91.31%
Best Recall: 91.31%


### RandomSearchCV (Random Search Cross-Validation): 
-  This method works by randomly sampling hyperparameter combinations from a specified distribution and evaluating the model's performance with each set of hyperparameters. This process is repeated multiple times, and the best-performing hyperparameters are selected as the optimal ones.

sklearn.model_selection.RandomizedSearchCV(estimator, param_distributions, *, n_iter=10, scoring=None, n_jobs=None, refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', random_state=None, error_score=nan, return_train_score=False)

In [10]:
# Thay đổi nhãn của các loại thời tiết không thuộc Clear hoặc Clouds thành 'Thời tiết xấu'
weather_df['Weather'] = weather_df['Weather'].replace(['Haze', 'Mist', 'Snow', 'Thunderstorm', 'Rain', 'Drizzle', 'Fog', 'Smoke'], 'Poor weather')

# Lấy các feature và nhãn đã chỉnh sửa
features = weather_df[['Temp', 'Humidity', 'Visibility', 'Wind speed', 'Clouds']]
labels = weather_df['Weather']

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Define the hyperparameter grid for RandomizedSearchCV
param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': ['balanced', 'balanced_subsample', None]
}

# Create the Random Forest classifier
rf_model = RandomForestClassifier(random_state=42)

# Create RandomizedSearchCV object
random_search = RandomizedSearchCV(rf_model, param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', random_state=42)
random_search.fit(X_train, y_train)

# Print the best hyperparameters
best_params = random_search.best_params_
print(f'Best Hyperparameters: {best_params}')

# Get the best model
best_rf_model = random_search.best_estimator_

# Make predictions on the test set using the best model
best_predictions = best_rf_model.predict(X_test)

# Evaluate the best model
best_precision = precision_score(y_test, best_predictions, average='weighted')
best_accuracy = accuracy_score(y_test, best_predictions)
best_recall = recall_score(y_test, best_predictions, average='weighted')

print(f'Best Precision: {best_precision * 100:.2f}%')
print(f'Best Accuracy: {best_accuracy * 100:.2f}%')
print(f'Best Recall: {best_recall * 100:.2f}%')


Best Hyperparameters: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_depth': 10, 'class_weight': None}
Best Precision: 90.17%
Best Accuracy: 90.74%
Best Recall: 90.74%


In [8]:
test = rf_model.predict([[ 293.16,	53,	10000	,2.29,	0],
                         [271.08	,62	,10000	,1.91,	0],
                         [279.33	,58	,10000	,0.52	,100],
                         [280.77,57,10000,3.60,0],
                         [269.27	,97	,141	,2.38	,100]])
test

c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array(['Clear', 'Clear', 'Clouds', 'Clear', 'Poor weather'], dtype=object)

In [9]:
test

array(['Clear', 'Clear', 'Clouds', 'Clear', 'Poor weather'], dtype=object)